In [1]:
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network

import pandas as pd 


network = Network.local()
client = AsyncClient(network)


async def sub_order(client, account_address) -> None:
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]


async def order_list(client, subaccount_id,market_id) -> None:
    orders = await client.fetch_subaccount_orders_list(subaccount_id=subaccount_id, market_id=market_id)
    return orders['orders']


async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(client, account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(client, i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    
    return df

In [2]:
from functions import marketId_to_ticker
from IPython.display import display
import pandas as pd

tickers = ["iMSTR/USDT PERP", "iNVDA/USDT PERP", "iCOIN/USDT PERP", "iTSLA/USDT PERP", "iHOOD/USDT PERP", "iMCD/USDT PERP", "iGOOGL/USDT PERP", "TRADFI/USDT PERP", "TTI/USDT PERP"]

address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

async def main():

    combined_df = pd.DataFrame()

    for ticker in tickers:
        market_id = await marketId_to_ticker(ticker)
        if market_id:
            print(f"Fetching orders for {ticker}...")
            df = await subfinder(account_address=address, market_id=market_id[0])
            
            df = df.iloc[:1]
            
            df['ticker'] = ticker
            
            display(df)
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            column_to_drop = ["orderSide","price", "quantity_in_$"]
            combined_df.drop(columns=column_to_drop, inplace=True)

    combined_df.to_csv(f"orders_{address}.csv", index=False)
    print(f"Combined CSV saved as orders_{address}.csv")

await main()

Fetching orders for iMSTR/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
0,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000027,306.2,1478.6398,iMSTR/USDT PERP


Fetching orders for iNVDA/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
0,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000002a,110.16,1363.23,iNVDA/USDT PERP


Fetching orders for iCOIN/USDT PERP...


...

Fetching orders for TRADFI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000001b,559.09,1397.725,TRADFI/USDT PERP


Fetching orders for TTI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000016,470.99,1412.97,TTI/USDT PERP


Combined CSV saved as orders_inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z.csv
